In [132]:
import os
import pandas as pd
import numpy as np
from pybaseball import batting_stats # Use this library to download the baseball stats

In [133]:
START = 2002
END = 2022

In [134]:
batting = batting_stats(START, END, qual=200) # Qual determines how many minimum plate appearances a batter can have

In [135]:
batting.to_csv("batting.csv") # Store this batting data into a CSV file

In [136]:
# Split batting data columns into groups for each player
# Filter groups to only include a player's data group that has more than 1 season of data
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1)

In [137]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
8,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7042,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
6535,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,...,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
6673,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
6988,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,...,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN,-2.9


In [138]:
# Setting up ML target
def next_season(player):
    # Split data up by player and for each player, backfill WAR value for next season as target
    player = player.sort_values("Season")
    player["Next WAR"] = player["WAR"].shift(-1)
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

/var/folders/51/kkgylr0s7vd8971v07bhrc040000gn/T/ipykernel_88743/762151536.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(next_season)


In [139]:
batting[["Name", "Season", "WAR", "Next WAR"]]

,Name,Season,WAR,Next WAR
5562,Alfredo Amezaga,2006,1.1,2.0
5006,Alfredo Amezaga,2007,2.0,1.2
5252,Alfredo Amezaga,2008,1.2,NaN
1169,Garret Anderson,2002,3.7,5.1
864,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
6002,Owen Miller,2022,0.7,NaN
4881,Andrew Vaughn,2021,-0.2,-0.5
3377,Andrew Vaughn,2022,-0.5,NaN
6620,Ha-seong Kim,2021,0.4,3.6


In [140]:
null_count = batting.isnull().sum()

In [141]:
null_count

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
xBA         6754
xSLG        6754
xwOBA       6754
L-WAR          0
Next WAR    1179
Length: 321, dtype: int64

In [142]:
complete_cols = list(batting.columns[null_count == 0]) # Select columns without any missing values
batting = batting[complete_cols + ["Next WAR"]].copy() # Select all complete columns and "Next War" column for batting

In [143]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next WAR
5562,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,107,113,143,109,63,0,0.188,0.256,1.1,2.0
5006,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,101,112,109,113,75,0,0.175,0.227,2.0,1.2
5252,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,101,123,111,64,0,0.178,0.244,1.2,NaN
1169,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,91,80,65,97,129,0,0.137,0.232,3.7,5.1
864,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,101,80,90,99,109,0,0.164,0.252,5.1,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,24655,2022,Owen Miller,CLE,25,130,424,472,103,70,...,111,97,131,100,83,340,0.188,0.266,0.7,NaN
4881,26197,2021,Andrew Vaughn,CHW,23,127,417,469,98,61,...,104,116,84,99,110,321,0.185,0.285,-0.4,-0.5
3377,26197,2022,Andrew Vaughn,CHW,24,134,510,555,138,92,...,106,111,94,100,104,419,0.201,0.291,-0.5,NaN
6620,27506,2021,Ha-seong Kim,SDP,25,117,267,298,54,32,...,99,59,137,96,88,201,0.216,0.303,0.5,3.6


In [144]:
batting.dtypes[batting.dtypes == "object"]

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [145]:
# Delete unnecessary String type columns - only player name and team should remain
del batting["Age Rng"]
del batting["Dol"]

In [147]:
# Assign numbers to each 3-letter team name abbreviation
batting["Team Code"] = batting["Team"].astype("category").cat.codes # Use pandas to categorically sort teams, and convert categories to a number

In [148]:
# Make copy of batting data because we're dropping any rows where "Next WAR" is empty but those rows could be useful in predicting future seasons
batting_full = batting.copy()
batting = batting.dropna().copy()

In [150]:
# Going to run feature selection to optimize model's accuracy
from sklearn.linear_model import Ridge # Ridge regression model (RRM)
from sklearn.feature_selection import SequentialFeatureSelector # Sequential feature selector (SFS)
from sklearn.model_selection import TimeSeriesSplit # Time series split (TSS)

# RRM documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
rr = Ridge(alpha=1) # Set lambda coefficient to 1
                    # Setting it HIGHER reduces overfitting because it penalizes ridge regression coefficients
                    # Setting it LOWER makes the model closer to a pure linear regression

# TSS documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html
split = TimeSeriesSplit(n_splits=3) # Split data into 3 parts and make predictions for each part in a time-series aware way

# SFS documentation: https://scikit-learn.org/dev/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html
sfs = SequentialFeatureSelector(rr, n_features_to_select=20, direction="forward", cv=split, n_jobs=4) # Pass in RRM, choose 20 features to select, use our time series
                                                                                                      # split to cross validate, use n_jobs to run faster

# Forward direction: Goes through all features and finds the best one, goes through remaining features and finds the next best one, so on until you have 20 features
# Backward direction: Goes through all features and removes the worst one, goes through remaining featuers and removes the next worst one, so on until you have 20 features
# --> Forward runs faster


In [151]:
# SFS shouldn't work with certain columns: target WAR, text columns, player ID (to avoid overfitting to particular player), and season (to avoid overfitting to particular season)
removed_columns = ["Next WAR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting.columns[~batting.columns.isin(removed_columns)] # Pick columns from batting that aren't in removed_columns

In [152]:
# Scale data so that mean = 0 and standard deviation = 1 for RRM to work effectively
from sklearn.preprocessing import MinMaxScaler # Min max scaler (MMS)

scaler = MinMaxScaler() # MMS is an agressive form of scaling that'll put all our values between 0 and 1 to avoid issues with finding ratios between columns
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns]) # Select all rows and columns from selected_columns

/var/folders/51/kkgylr0s7vd8971v07bhrc040000gn/T/ipykernel_88743/2309204606.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.34615385 0.38461538 0.42307692 ... 0.19230769 0.15384615 0.23076923]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns]) # Select all rows and columns from selected_columns
/var/folders/51/kkgylr0s7vd8971v07bhrc040000gn/T/ipykernel_88743/2309204606.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.73504274 0.74358974 0.95726496 ... 0.11965812 0.69230769 0.60683761]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns]) # Select all rows and columns from selected_columns
/var/f

In [153]:
# Some columns have been scaled
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next WAR,Team Code
count,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,...,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000
mean,5366.785830,2011.163229,0.360600,0.652755,0.478666,0.480943,0.365973,0.290481,0.399279,0.103459,...,0.403164,0.410923,0.511026,0.478646,0.172991,0.498932,0.545898,0.322040,1.793130,0.474128
std,5133.255295,5.612014,0.147476,0.255929,0.242481,0.262290,0.182585,0.138786,0.171732,0.105891,...,0.131213,0.121082,0.130359,0.133992,0.273858,0.137180,0.120701,0.122153,1.981035,0.305105
min,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.100000,0.000000
25%,1131.500000,2006.000000,0.269231,0.478632,0.275180,0.257785,0.211207,0.179245,0.258621,0.043478,...,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.234177,0.350000,0.205882
50%,3531.000000,2011.000000,0.346154,0.709402,0.505396,0.508651,0.370690,0.283019,0.379310,0.086957,...,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,0.303797,1.500000,0.470588
75%,9015.000000,2016.000000,0.461538,0.871795,0.688849,0.710208,0.508621,0.391509,0.517241,0.130435,...,0.488722,0.483146,0.594203,0.564626,0.346411,0.591489,0.625551,0.392405,2.900000,0.735294
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [154]:
# After scaling data, we can apply SFS
sfs.fit(batting[selected_columns], batting["Next WAR"]) # Fitting will pick the 20 predictors that give us the greatest accuracy in our RRM

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [155]:
# Extract list of predictors
predictors = list(selected_columns[sfs.get_support()]) # sfs.get_support() returns a big array of Trues and Falses where Trues are columns we want to select
                                                       # Index selected_columns with method call to give us pandas index of predictor columns
                                                       # Call list() to create readable list of predictor columns

In [156]:
# Backtest function will generate our predictions for us

# Cross validation will split the dataset up into groups while still making predictions on the entire dataset
# (i.e. Use groups 2 through n to train the algorithm to predict group 1, use groups 1 and 3 through n to train the algorithm to predict group 2, etc.)
# Cross validation doesn't work on time series data because we can't use future data to predict past data --> we only want to use past data to predict future data
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = [] # Each element in the list is the predictions for a single season

    years = sorted(data["Season"].unique()) # Find all unique seasons and sort them in order

    # Each time we go through this loop, we'll use historical data to predict a single season starting with predicting the 2007 season (hence start=5) using 2002-2006
    # to have enough training data and then using 2002-2007 to predict 2008, etc.
    for i in range(start, len(years), step):
        current_year = years[i];

        train = data[data["Season"] < current_year] # Choose training set to be data before current_year
        test = data[data["Season"] == current_year] # Choose test set to be current_year

        model.fit(train[predictors], train["Next WAR"]) # Fit model using training predictors and our target that we're trying to predict (Next WAR)

        preds = model.predict(test[predictors]) # Generate predictions on the test set
        preds = pd.Series(preds, index=test.index) # Turn default numpy array from predict() to pandas series for easier usability
        combined = pd.concat([test["Next WAR"], preds], axis=1) # Combine actual values (test["Next WAR"]) with predictions (preds)
                                                                # axis=1 allows us to have 2 separate columns
        combined.columns = ["Actual", "Prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions, axis=0)

In [157]:
predictions = backtest(batting, rr, predictors)

In [158]:
predictions

,Actual,Prediction
5006,1.2,1.405835
1925,1.4,0.716105
3102,-0.1,0.457908
5797,0.6,0.979155
1109,4.8,2.214873
...,...,...
1914,2.2,2.752087
5875,0.8,2.084895
7032,0.7,1.583738
4881,-0.5,1.819861


In [159]:
# Create a summary statistic to make an error metric
from sklearn.metrics import mean_squared_error # Mean squared error (MSE)

# MSE gives us a single number that tells us how high the error is in the model by subtracting Prediction from Actual for each row, squaring the difference, and then
# finding the average squared difference across all rows
mean_squared_error(predictions["Actual"], predictions["Prediction"]) # 2.7363675228708013

2.7363675228708013

In [160]:
batting["Next WAR"].describe() # Ideally, we'd like the MSE (~2.74) to be lower than the standard deviation (~1.98) at the minimum

count    5575.000000
mean        1.793130
std         1.981035
min        -3.100000
25%         0.350000
50%         1.500000
75%         2.900000
max        11.900000
Name: Next WAR, dtype: float64

In [161]:
# To improve the algorithm, we're going to give it information about each player's performance in previous seasons

# player_history takes in a player's data from a single season
def player_history(df):
    df = df.sort_values("Season")

    df["Player Season"] = range(0, df.shape[0]) # Indicates which season the player is playing
    df["WAR Correlation"] = list(df[["Player Season", "WAR"]].expanding().corr().loc[(slice(None), "Player Season"), "WAR"]) # WAR correlation
        # expanding() creates different groups of the dataframe (first row, first 2 rows, first 3 rows, etc.)
        # corr() allows us to find the correlation between player season and WAR within each group of expanding()
        # Using loc() and list() gives us a list of correlations between the season and the player's WAR for each season
    df["WAR Correlation"].fillna(1, inplace=True) # Fill missing values with 1 to imply 1-to-1 correlation between season and WAR

    df["WAR Differential"] = df["WAR"] / df["WAR"].shift(1) # Ratio between current season's WAR and previous season's WAR (shifted back 1)
    df["WAR Differential"].fillna(1, inplace=True) # Fill missing values with 1 to imply 1-to-1 correlation between season and WAR
    df["WAR Differential"][df["WAR Differential"] == np.inf] = 1 # Replace any infinite values with 1

    return df

batting = batting.groupby("IDfg", group_keys=False).apply(player_history)
    # Splits batting dataframe up into groups by player, and for each player, it calls player_history() and passes in the player's data

/var/folders/51/kkgylr0s7vd8971v07bhrc040000gn/T/ipykernel_88743/2677383305.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["WAR Correlation"].fillna(1, inplace=True) # Fill missing values with 1 to imply 1-to-1 correlation between season and WAR
/var/folders/51/kkgylr0s7vd8971v07bhrc040000gn/T/ipykernel_88743/2677383305.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will

In [162]:
# group_averages() will find averages across a whole season for all players and compare them with how each player performed
def group_averages(df):
    return df["WAR"] / df["WAR"].mean() # Also helps to correct against seasons where a player played less games due to lockouts so that their lower WAR due to
                                        # less games doesn't impact their effectiveness on the field statistically

In [163]:
batting["Season WAR"] = batting.groupby("Season", group_keys=False).apply(group_averages) # Creates 1 group for each season, finds the average for a season,
                                                                                          # and how each player compared to that average

/var/folders/51/kkgylr0s7vd8971v07bhrc040000gn/T/ipykernel_88743/1186079255.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting["Season WAR"] = batting.groupby("Season", group_keys=False).apply(group_averages) # Creates 1 group for each season, finds the average for a season,


In [164]:
new_predictors = predictors + ["Player Season", "WAR Correlation", "WAR Differential", "Season WAR"]

In [165]:
predictions = backtest(batting, rr, new_predictors)

In [166]:
mean_squared_error(predictions["Actual"], predictions["Prediction"]) # 2.679213472448685

2.679213472448685

In [167]:
pd.Series(rr.coef_, index=new_predictors).sort_values() # Identifying influential predictors

Age                -2.711987
BABIP              -1.939334
WAR                -1.842281
SLG+               -1.380470
Soft%+             -1.322812
BU                 -1.126876
SO                 -0.896729
PH                 -0.745613
WPA                -0.559244
CH%                -0.288173
wCH                -0.288016
WAR Differential   -0.283706
CB%                -0.276117
Pull%+             -0.218182
WAR Correlation    -0.137705
Player Season       0.000036
IFH                 0.634505
Oppo%               0.694873
Spd                 0.768871
OBP+                0.828260
SB                  0.961873
IBB                 2.059969
Hard%+              2.442866
Season WAR          3.190842
dtype: float64

In [173]:
diff = predictions["Actual"] - predictions["Prediction"]

In [174]:
diff

5006   -0.222652
1925    0.909449
3102   -0.346150
5797   -0.386085
1109    2.842180
          ...   
1914   -0.450297
5875   -1.166754
7032   -0.668185
4881   -2.065503
6620    2.663649
Length: 4127, dtype: float64

In [175]:
merged = predictions.merge(batting, left_index=True, right_index=True)

In [176]:
merged["Differential"] = (predictions["Actual"] - predictions["Prediction"]).abs() # Table now includes a column showing the difference between the actual and predicted WAR

In [177]:
merged[["IDfg", "Season", "Name", "WAR", "Next WAR", "Differential"]].sort_values(["Differential"])

,IDfg,Season,Name,WAR,Next WAR,Differential
6023,4403,2013,Erik Kratz,0.246835,1.1,0.001375
6973,17696,2021,Kevin Newman,0.170886,1.2,0.001389
1190,15172,2019,Tim Anderson,0.481013,2.3,0.002103
3266,1286,2008,Michael Young,0.348101,2.6,0.003986
2082,5887,2013,John Jaso,0.234177,0.6,0.004329
...,...,...,...,...,...,...
3823,1875,2009,Josh Hamilton,0.278481,8.4,6.457327
871,9166,2010,Buster Posey,0.443038,9.8,6.526769
3245,5631,2010,Matt Kemp,0.196203,8.3,6.526948
451,15640,2021,Aaron Judge,0.544304,11.1,7.311515
